In [2]:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tqdm.auto import tqdm


from statsmodels.tsa.arima_model import ARIMA

In [3]:
DATA_PATH = './data'

In [4]:
train_x = pd.read_csv(DATA_PATH  + "/train_x_df.csv")
train_y = pd.read_csv(DATA_PATH  + "/train_y_df.csv")
test_x = pd.read_csv(DATA_PATH  + "/test_x_df.csv")


In [5]:
train_x["is_x"] = 1
train_y["is_x"] = 0

In [7]:

train_x_y = [train_x, train_y]

In [10]:

train_list = [x.set_index('sample_id') for x in train_x_y]

train_z = pd.concat(train_list, axis=0).rename_axis('sample_id').reset_index()


In [11]:
train_z

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,is_x
0,0,0,9,0.983614,0.983614,0.983128,0.983246,0.001334,10.650987,0.009855,0.000848,6.771755,1
1,0,1,9,0.983245,0.983612,0.982453,0.982693,0.001425,11.375689,0.016137,0.000697,5.565188,1
2,0,2,9,0.982694,0.983612,0.982403,0.983002,0.001542,12.301942,0.014166,0.000905,7.225459,1
3,0,3,9,0.983009,0.984848,0.983009,0.984486,0.002520,20.134695,0.021557,0.001171,9.353000,1
4,0,4,9,0.984233,0.984606,0.983612,0.984164,0.002818,22.515448,0.021434,0.001799,14.372534,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11491495,7660,115,8,1.002662,1.003382,1.002606,1.002717,0.957148,173.131668,0.510148,0.476201,86.143784,0
11491496,7660,116,8,1.002551,1.002606,1.001608,1.001830,1.537946,277.939087,0.465787,1.061331,191.801056,0
11491497,7660,117,8,1.001830,1.002384,1.001497,1.001608,1.203197,217.364487,0.637684,0.971337,175.472198,0
11491498,7660,118,8,1.001608,1.001941,1.001109,1.001386,1.252859,226.264069,1.003660,0.449899,81.251137,0


In [12]:
# vwap과, open에서 vwap을 뺀 값인 diff를 계산합니다.
def make_vwap_and_diff(df):

    # 1) VAWP 계산

    # 일반적인 VWAP 공식에서 volume을 그대로 사용하지만, 여러번의 시도를 통해 tb_base_av 와 volume을 더했을 때 가장 좋은 volume이 나온다고 판단하영 사용하였습니다.
    df["volume_tb_base_av"] = df["tb_base_av"] + df["volume"]

    # open하나만을 사용하기 보다는 open(시가), high(고가), low(저가) 3개의 평균을 price로 사용하였습니다.
    df['volume_price'] = ((df['open'] + df['high'] + df['low']) / 3) * df['volume_tb_base_av']

    # price와 volume의 곱의 합을 구해줍니다.
    df['volume_price_sum'] = df.groupby(['sample_id'])['volume_price'].apply(lambda x: x.cumsum())

    # volume의 합을 구해줍니다.
    df['volume_sum'] = df.groupby(['sample_id'])['volume_tb_base_av'].apply(lambda x: x.cumsum())

    # 2 변수의 나눗셈을 통해 vwap을 계산해줍니다.
    df['vwap'] = df['volume_price_sum'] / df['volume_sum']


    # 2) diff 계산
    # 매도수익이 open을 통해 이루어진다고 알려져있기 때문에 open에서 vwap을 뺀 값을 diff로 사용했습니다.
    df["diff"] = df["open"] - df["vwap"]


    return df
    

In [13]:
# 데이터 프레임에서 sample_id 에 따른 open을 반환하는 함수입니다.
def get_open(df,sample_id):
    
    return df[df["sample_id"] == sample_id]['open'].values
    

In [14]:
# 데이터 프레임에서 sample_id 에 따른 VWAP을 반환하는 함수입니다.
def get_vwap(df,sample_id):
    
    return df[df["sample_id"] == sample_id]['vwap'].values
    

In [15]:
# 데이터 프레임에서 sample_id 에 따른 diff 반환하는 함수입니다.
def get_diff(df,sample_id):
    
    return df[df["sample_id"] == sample_id]['diff'].values
    

In [16]:
# 데이터 프레임에서 sample_id 에 따른 rsi 반환하는 함수입니다.
def get_rsi(df,sample_id):
    
    return df[df["sample_id"] == sample_id]['rsi'].values
    

In [17]:
# 데이터 프레임에서 sample_id 에 따른 col_name 값을 반환하는 함수입니다.
def get_series(df,sample_id, col_name):
    
    return df[df["sample_id"] == sample_id][col_name].values
    

### 2) RSI
상대강도지수(relative strength index) 는 가격의 상승압력과 하락압력 간의 상대적인 강도를 나타냅니다.

<br>

트레이딩에서 사용되며, RSI 값이 30보다 작으면 초과매도로 판단하고, RSI 값이, 70 보다 크면 초과매수 상태로 판단합니다.

<br>

![](https://raw.githubusercontent.com/skyepodium/bit-traider-image/master/image/rsi_formula.png)

<br>

RSI는 최고점과, 최하점을 찾기 쉽다는 장점이 있습니다. 다만, 일명 박스권으로 천정과 바닥이 제대로 형성되지 않은 시장(RSI가 50근처 유지)에서는 유용하지 못합니다.

코인 데이터의 RSI 그래프를 그려보면 RSI 0 ~ 100 사이를 급변동 합니다.

<br>
저는 조금 더 안정적으로 투자하기 위해 65 초과인 상태를 초과매수국면으로 판단하고, 해당 시점 이후 50분동안은 투자하지 않도록 정해주었습니다.




In [13]:
# RSI를 만들어줍니다.
# 상승분, 하락분의 평균은 일반적으로 14일을 기준으로 생성합니다.
def make_rsi(df, period=14):

    # 전일 대비 상승분을 계산해줍니다. - 상승분이 0보다 크면 상승분을 넣고, 0보다 작거나 같으면 0을 넣어줍니다.
    df["U"] = np.where(df.groupby(["sample_id"])["open"].diff(1) > 0, df.groupby(["sample_id"])["open"].diff(1), 0)

    # 전일 대비 하락분을 계산해줍니다. - 하락분이 0보다 작으면 하락분 * -1을 넣고, 0보다 크거나 같으면 0을 넣어줍니다.
    df["D"] = np.where(df.groupby(["sample_id"])["open"].diff(1) < 0, df.groupby(["sample_id"])["open"].diff(1) *(-1), 0)


    # 전일 대비 상승분의 평균을 계산해줍니다.
    ud_df = pd.DataFrame()
    ud_df["sample_id"] = df["sample_id"]
    ud_df["U"] = df["U"]
    ud_df["D"] = df["D"]

    # 상승분의 14일 평균을 구해줍니다.
    df["AU"] = ud_df.groupby(["sample_id"])["U"].rolling( window=period, min_periods=period ).mean().reset_index()["U"]
    # 하락분의 14일 평균을 구해줍니다.
    df["AD"] = ud_df.groupby(["sample_id"])["D"].rolling( window=period, min_periods=period ).mean().reset_index()["D"]


    # AU / (AU + AD) 의 백분율을 RSI 로 계산해줍니다.
    RSI = df["AU"] / (df["AU"] + df["AD"]) * 100
    
    df["rsi"] = RSI
    
    return df
    

# 6. 전처리 🪄

전처리 단계에서는 5번에서 작성한 함수를 train, test 데이터에 적용하여 피쳐 생성을 진행합니다.

In [14]:
# 1. train, test의 sample_id 목록을 저장합니다.
TRAIN_SAMPLE_ID_LIST = train_x["sample_id"].unique().tolist()
TEST_SAMPLE_ID_LIST = test_x["sample_id"].unique().tolist()


In [15]:
# 2. VWAP, diff 를 만들어줍니다.
test_x = make_vwap_and_diff(test_x)
train_z = make_vwap_and_diff(train_z)


In [16]:
# 3. rsi 를 만들어줍니다.
test_x = make_rsi(test_x, 14)
train_z = make_rsi(train_z, 14)


In [17]:
# 4. train x와 y를 분리합니다.
train_x = train_z[train_z["is_x"] == 1]
train_y = train_z[train_z["is_x"] == 0]

split_drop_cols = ["is_x"]

train_x = train_x.drop(columns=split_drop_cols, axis=1)
train_y = train_y.drop(columns=split_drop_cols, axis=1)


# 7. 모델 학습 🤑

### 1) 모델
모델은 diff (open - vwap)을 ARIMA를 통해 학습하여 생성했습니다. 

<br>

### 2) 제약조건
vwap, rsi의 마지막 값을 제약조건의 기준으로 사용했습니다.
vwap이 1보다 크면 open보다 vwap이 크다는 의미로 하향세에 접어들었다고 판단 투자하지 않았습니다.
(x의 open 의 마지막 값은 1입니다.)

<br>

rsi의 값이 65보다 크면 초과 매수 상태라고 판단하여 투자하지 않았습니다. 기본적으로 70 초과로 판단하는데 임의로 바꿀수 있는 값으로 조금 낮춰서 65로 시도했을때가 제일 좋아서 사용했습니다.

<br>

### 3) ARIMA와 p d q

개인적으로 ARIMA 모델의 order(p, d, q)에 투자를 많이 했었습니다.

ARIMA 모델의 AIC (아카이케 정보 기준) 가 가장 나오는 order를 brute force로 찾아보았지만, 

2시간 30분 ~ 3시간이 걸림에도 불구하고, 점수가 오히려 낮아져서, 여러번의 시도로 4, 0, 1 이 제일 좋다고 판단했습니다.

pac, acf 분석을 통해 AR 2 모델이라고 판단했습니다. 분석에는 다음 영상을 참고했습니다. (https://www.youtube.com/watch?v=-vSzKfqcTDg&t=360s)


In [18]:
result = []

for sample_id in tqdm(TEST_SAMPLE_ID_LIST):

    # 1. 데이터 로드
    # 1) diff - 학습에 사용
    diff_x = get_diff(test_x, sample_id)

    # 2) vwap - 보조 지표로 사용
    vwap_series = get_vwap(test_x, sample_id)

    # 3) rsi - 보조 지표로 사용
    rsi_series = get_rsi(test_x, sample_id)



    # 2. ARIMA
    # 1) 모델 정의
    ARIMA_MODEL = {}
    ARIMA_MODEL_FIT = {}

    # 2) AR 모델 적용
    try:
      ARIMA_MODEL = ARIMA(diff_x, order = (4,0,1))
      ARIMA_MODEL_FIT = ARIMA_MODEL.fit(trend = 'nc', full_output = True, disp = True)

    # 3) 수렴하지 않을 경우 p d q 를 1, 1, 0으로 사용
    except:
      ARIMA_MODEL = ARIMA(diff_x, order = (1,1,0))
      ARIMA_MODEL_FIT = ARIMA_MODEL.fit(trend = 'nc', full_output = True, disp = True)

    # 4) ARIMA 예측
    ARIMA_FORECAST  = ARIMA_MODEL_FIT.predict(1,120, typ='levels')



    # 3. 데이처 처리
    # 1) 최대 부분인 인덱스를 찾는데 해당 시점에 매도를 진행합니다.
    sell_time = np.argmax(ARIMA_FORECAST)

    # 2) 최대값을 찾습니다.
    max_val = np.max(ARIMA_FORECAST)
    
    # 3) vwap의 마지막 값을 가져옵니다.
    vwap_last_val = vwap_series[1379]

    rsi_last_val = rsi_series[1379]



    # 4. 투자 전략
    buy_quantity = 0

    # 1) 최대값이 0 보다 크면 가격이 vwap 보다 크다는 의미로, 투자합니다.
    if  max_val > 0:
        buy_quantity = 1


    # 2) 만약 vwap 마지막 값이, 1보다 크면 가격이 1보다 작다는 의미로 하향세이기 때문에 투자하지 않습니다.
    if vwap_last_val > 1 and sell_time < 50:
        buy_quantity = 0

    # 3) 만약 rsi의 값이 65 보다 크면, 초과매수 상태로 판단하여 투자하지 않습니다.
    if rsi_last_val > 65 and sell_time < 50:
        buy_quantity = 0



    # 5. 결과
    result_list = [
                   sample_id,
                   buy_quantity,
                   sell_time
                  ]

    result.append(result_list)
    

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:568: 

# 8. 제출 🎉

In [19]:
# 1. 학습 결과를 데이터 프레임으로 만듭니다.

submit_columns = [
                  "sample_id", 
                  "buy_quantity", 
                  "sell_time"
                  ]


submit = pd.DataFrame(data=result, columns=submit_columns)


In [20]:
# 2. 결과 데이터 프레임 확인

submit.head(10)


,sample_id,buy_quantity,sell_time
0,7661,0,47
1,7662,1,62
2,7663,0,11
3,7664,0,48
4,7665,1,46
5,7666,1,70
6,7667,1,119
7,7668,1,82
8,7669,1,109
9,7670,1,66


In [21]:
# 3. 투자 개수 확인

submit[submit["buy_quantity"] == 1].shape[0]


420

In [22]:
# 4. sell_time 50미만에서 구매하는 개수 확인
cond1 = (submit["buy_quantity"] == 1)
cond2 = (submit["sell_time"] < 50)

submit[cond1 & cond2].shape[0]


84

In [23]:
# 5. 제출

In [24]:
# 파일의 이름을 지정해줍니다.
FILE_NAME = "/0603_ARIMA_DIFF_VWAP_RSI_65_UNDER_50_SUBMIT.csv"


In [25]:
# 제출경로에 파일을 생성해줍니다.
RESULT_PATH = SUBMIT_PATH + FILE_NAME

submit.to_csv(RESULT_PATH, index=False)


# 9. 안정적 모델인지 어떻게 확인? 🤔
시즌 1을 경험하고, 최대한 안정적인 모델을 생성하기 위해 노력했습니다. 

다만, 내가 안정적이라고 생각해도, 객관적인 지표가 없다보니 얼마나 균형잡혀있는지 확인하기가 어려웠습니다.

이를 위해 public score를 다음과 같이 활용했습니다.

<br>

예를 들어, 
- open에 vwap을 섞어 투자개수가 줄었음에도 점수가 올라가는 현상
- vwap, rsi등 보조지표를 통해 sell_time 10, 20.. 50 미만은 투자하지 않기로 결정했음에도 점점 점수가 올라가는 현상
- rsi 초과매수로 상태를 70 초과가 아닌, 65 초과로, 더 안정적으로 결정해도 점수가 상승하는 현상
<br>

이렇게, 제약사항을 통해 보수적으로 투자했음에도 불구하고, 점수가 올라가는 경우에 집중하여, 해당 피쳐가 안정적으로 작용함을 판단했습니다.


# 10. 여러 아이디어 🥴
65번이라는 조금 많은 제출을 통해 여러 시도를 진행했습니다. 저의 개인적인 결과는 좋지 않았습니다만, 혹시나 이후 대회에서 다른 분들에게 조금이라도 도움이 될 수 있을까 해서 남기게 되었습니다.

<br>

### 1) RANDOM BOX 분류 모델
시계열 대회이지만, classification으로 생각해보았습니다.

결국에 제가 선택하는것은 120개중 1개이기 때문에 120개의 상자 중 1개를 선택했을때 그 값이 1보다 클 확률을 제가 얻을 수 있는 기대값이라고 생각했습니다. 

개인적으로 가장 창의적이라고 생각했지만, 잘 안되었습니다. 더 지니어스의 콩픈패스에 영감을 받았습니다.


<br>

### 2) 혼합 데이터
open에 vwap을 섞은것에 더하여, RSI, 이동평균선등을 모두 섞어 보았습니다. 생각보다 점수가 정말 좋았지만, vwap 한개를 섞은것보다는 아쉽다고 생각하여 사용하지는 않았습니다.

<br>



# 11. 다른 모델 사용여부 😁
ARIMA 이외에도 prophet, neural prophet, LSTM등 다양한 모델을 사용해보았습니다. 

다만, 더 좋은 결과를 주지 않았다고 판단했습니다.

또한, 위 모델들의 결과를 bagging하는 방법을 사용해보았지만, ARIMA 단 하나만을 사용했을때보다 예측 성능이 낮아진다고 판단하여 사용하지 않게 되었습니다.

